In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [12]:
# List for holding lat_lngs and cities
lat_lngs = []
in_cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    in_city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if in_city not in in_cities:
        in_cities.append(in_city)

# Print the city count to confirm sufficient count
len(in_cities)

602

In [13]:
def get_weather_json(city, api_key, units):
    
    url = "http://api.openweathermap.org/data/2.5/weather"
    r = requests.get(url, params={
        "appid": api_key,
        "q": city,
        "units": units
    })
    if r.status_code == 200:
        r_json = r.json()
        return r_json
    else:
        return None
    

In [14]:
#in_cities = ["austin", "houston", "trontsrad;", "dallas", "fort worth"]
out_cities = []
unit = "Imperial"
for in_city in in_cities:
    out_city = get_weather_json(in_city, api_key, unit)
    if out_city:
        out_cities.append(out_city)

In [9]:
#out_cities

[{'coord': {'lon': -97.74, 'lat': 30.27},
  'weather': [{'id': 701,
    'main': 'Mist',
    'description': 'mist',
    'icon': '50n'}],
  'base': 'stations',
  'main': {'temp': 60.8,
   'feels_like': 58.55,
   'temp_min': 59,
   'temp_max': 63,
   'pressure': 1011,
   'humidity': 88},
  'visibility': 8047,
  'wind': {'speed': 8.05, 'deg': 140},
  'clouds': {'all': 90},
  'dt': 1577523902,
  'sys': {'type': 1,
   'id': 3344,
   'country': 'US',
   'sunrise': 1577539572,
   'sunset': 1577576307},
  'timezone': -21600,
  'id': 4671654,
  'name': 'Austin',
  'cod': 200},
 {'coord': {'lon': -95.37, 'lat': 29.76},
  'weather': [{'id': 741, 'main': 'Fog', 'description': 'fog', 'icon': '50n'}],
  'base': 'stations',
  'main': {'temp': 63.43,
   'feels_like': 63.19,
   'temp_min': 62.01,
   'temp_max': 64.99,
   'pressure': 1012,
   'humidity': 100},
  'visibility': 402,
  'wind': {'speed': 8.63, 'deg': 128},
  'clouds': {'all': 90},
  'dt': 1577524047,
  'sys': {'type': 1,
   'id': 4850,
   'c

In [15]:
pd.DataFrame(out_cities)
cleaned_out_cities = [{
    "name": d["name"],
    "max temp": d["main"]["temp_max"],
    "cloudiness": d["clouds"]["all"],
    "country": d["sys"]["country"],
    "date": d["dt"],
    "humidity": d["main"]["humidity"],
    "lat": d["coord"]["lat"],
    "long": d["coord"]["lon"],
    "wind spd": d["wind"]["speed"]
} for d in out_cities]
pd.DataFrame(cleaned_out_cities)

,name,max temp,cloudiness,country,date,humidity,lat,long,wind spd
0,Geraldton,19.40,5,CA,1577524603,85,49.72,-86.95,8.05
1,Rikitea,76.41,100,PF,1577524604,88,-23.12,-134.97,11.72
2,Aklavik,-34.60,5,CA,1577524604,65,68.22,-135.01,3.44
3,Severo-Kurilsk,28.90,100,RU,1577524604,97,50.68,156.12,40.51
4,Mataura,53.01,87,NZ,1577524604,73,-46.19,168.86,11.99
...,...,...,...,...,...,...,...,...,...
537,Bar Harbor,37.99,90,US,1577524707,79,44.39,-68.20,4.70
538,Fayaoue,78.35,10,NC,1577524707,85,-20.65,166.53,25.55
539,Batemans Bay,84.99,79,AU,1577524707,81,-35.71,150.18,4.00
540,Hauterive,32.00,90,CA,1577524707,85,49.18,-68.27,9.17
